# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparametere Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.9s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.5s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.8s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +2.1s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-05 12:13:40,671	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-05 12:13:40,693	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_e5eb6cf222e19ba5.zip' (1.90MiB) to Ray cluster...
2022-01-05 12:13:42,622	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_e5eb6cf222e19ba5.zip'.


(anyscale +17.2s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_MWpJ7cXMEna9qF7arw5kj4s8
(anyscale +17.2s) URL for head node of cluster: https://session-mwpj7cxmena9qf7arw5kj4s8.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

In [4]:
ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [5]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [6]:
 # Specify the hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="50%" width="60%"> 

In [7]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=None) == Status ==
(run pid=None) Current time: 2022-01-05 12:13:50 (running for 00:00:00.13)
(run pid=None) Memory usage on this node: 2.7/62.0 GiB
(run pid=None) Using FIFO scheduling algorithm.
(run pid=None) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/91.58 GiB objects
(run pid=None) Result logdir: /home/ray/ray_results/train_func_model_2022-01-05_12-13-49
(run pid=None) Number of trials: 4/4 (4 PENDING)
(run pid=None) +------------------------------+----------+-------+-----------+-------------+-------------+
(run pid=None) | Trial name                   | status   | loc   |       eta |   max_depth |   subsample |
(run pid=None) |------------------------------+----------+-------+-----------+-------------+-------------|
(run pid=None) | train_func_model_fede3_00000 | PENDING  |       | 0.0628575 |           1 |    0.709828 |
(run pid=None) | train_func_model_fede3_00001 | PENDING  |       | 0.0202422 |           3 |    0.637472 |
(run pid=None) | train

(ImplicitFunc pid=None, ip=172.31.49.22) 2022-01-05 12:13:53,224	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=None, ip=172.31.49.130) 2022-01-05 12:13:53,220	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=None, ip=172.31.44.136) 2022-01-05 12:13:53,216	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=None) 2022-01-05 12:13:53,377	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=None, ip=172.31.49.130) 2022-01-05 12:13:55,338	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(ImplicitFunc pid=None, ip=172.31.44.136) 2022-01-05 12:13:55,334	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(ImplicitFunc pid=None, ip=172.31.4

(run pid=None) Result for train_func_model_fede3_00000:
(run pid=None)   date: 2022-01-05_12-13-56
(run pid=None)   done: false
(run pid=None)   experiment_id: 51c4446235134e45bb45dbea6db44bc8
(run pid=None)   hostname: ip-172-31-49-130
(run pid=None)   iterations_since_restore: 1
(run pid=None)   node_ip: 172.31.49.130
(run pid=None)   pid: 1127
(run pid=None)   time_since_restore: 2.9892845153808594
(run pid=None)   time_this_iter_s: 2.9892845153808594
(run pid=None)   time_total_s: 2.9892845153808594
(run pid=None)   timestamp: 1641413636
(run pid=None)   timesteps_since_restore: 0
(run pid=None)   train-error: 0.082601
(run pid=None)   train-logloss: 0.650691
(run pid=None)   training_iteration: 1
(run pid=None)   trial_id: fede3_00000
(run pid=None)   
(run pid=None) 2022-01-05 12:13:56,164	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=None) 2022-01-05 12:13:56,165	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the 

(ImplicitFunc pid=None, ip=172.31.49.22) 2022-01-05 12:14:02,697	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.51 seconds (7.35 pure XGBoost training time).
(ImplicitFunc pid=None, ip=172.31.49.130) 2022-01-05 12:14:02,708	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.53 seconds (7.36 pure XGBoost training time).
(ImplicitFunc pid=None, ip=172.31.44.136) 2022-01-05 12:14:02,701	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.53 seconds (7.36 pure XGBoost training time).
(ImplicitFunc pid=None) 2022-01-05 12:14:02,711	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 9.37 seconds (7.01 pure XGBoost training time).


(run pid=None) Result for train_func_model_fede3_00000:
(run pid=None)   date: 2022-01-05_12-14-00
(run pid=None)   done: false
(run pid=None)   experiment_id: 51c4446235134e45bb45dbea6db44bc8
(run pid=None)   hostname: ip-172-31-49-130
(run pid=None)   iterations_since_restore: 3
(run pid=None)   node_ip: 172.31.49.130
(run pid=None)   pid: 1127
(run pid=None)   time_since_restore: 7.52846360206604
(run pid=None)   time_this_iter_s: 1.9368565082550049
(run pid=None)   time_total_s: 7.52846360206604
(run pid=None)   timestamp: 1641413640
(run pid=None)   timesteps_since_restore: 0
(run pid=None)   train-error: 0.080844
(run pid=None)   train-logloss: 0.577872
(run pid=None)   training_iteration: 3
(run pid=None)   trial_id: fede3_00000
(run pid=None)   
(run pid=None) Result for train_func_model_fede3_00002:
(run pid=None)   date: 2022-01-05_12-14-02
(run pid=None)   done: true
(run pid=None)   experiment_id: 466c75627f7a4ba2ad180bd94309e2e7
(run pid=None)   experiment_tag: 2_eta=0.001

(run pid=None) 2022-01-05 12:14:02,842	INFO tune.py:626 -- Total run time: 13.01 seconds (11.99 seconds for the tuning loop).


In [8]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.02024216327854506, 'subsample': 0.6374720067304913, 'max_depth': 3}


In [9]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
fede3_00000,0.405660,0.057996,0.008818,True,None,None,10,51c4446235134e45bb45dbea6db44bc8,2022-01-05_12-14-02,1641413642,...,10,"0_eta=0.062858,max_depth=1,subsample=0.70983",approx,binary:logistic,"[logloss, error]",0.062858,0.709828,1,1,1
fede3_00001,0.547695,0.024605,0.008799,True,None,None,10,9aa02f1c5cfe4d00a75faa4b3467594d,2022-01-05_12-14-02,1641413642,...,10,"1_eta=0.020242,max_depth=3,subsample=0.63747",approx,binary:logistic,"[logloss, error]",0.020242,0.637472,3,1,1
fede3_00002,0.684813,0.079086,0.006039,True,None,None,10,466c75627f7a4ba2ad180bd94309e2e7,2022-01-05_12-14-02,1641413642,...,10,"2_eta=0.0011991,max_depth=1,subsample=0.72164",approx,binary:logistic,"[logloss, error]",0.001199,0.721639,1,1,1
fede3_00003,0.362479,0.026362,0.006740,True,None,None,10,e7dff74f5eca4bca93d701996fe4dd0c,2022-01-05_12-14-02,1641413642,...,10,"3_eta=0.061537,max_depth=3,subsample=0.5605",approx,binary:logistic,"[logloss, error]",0.061537,0.560502,3,1,1


In [10]:
ray.shutdown()

---

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](Distributed XGBoost on Ray)